# Projections

In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 1_000

### Categories

In [2]:
SKATER_CATEGORIES = ['goals', 'assists', 'plus_minus', 'powerplay_points', 'shots_on_goal', 'hits', 'blocks']
GOALIE_CATEGORIES = ["wins", "save_percentage", "goals_against_average", "saves", "shutouts"]

### Players

In [3]:
df = pd.read_csv("../data/adp-yahoo_com.csv")
df = df[["team", "name", "position"]]

df.head()

,team,name,position
0,Edm,Connor McDavid,C
1,Col,Nathan MacKinnon,C
2,Edm,Leon Draisaitl,"C,LW"
3,NYR,Artemi Panarin,LW
4,Was,Alex Ovechkin,LW


### Projection Data

In [4]:
prj = pd.concat([
    pd.read_csv("../data/projections-cbs_com.csv"), 
    pd.read_csv("../data/projections-dailyfaceoff_com.csv"), 
    pd.read_csv("../data/projections-yahoo_com.csv")
])

# fix weird assists issue
prj["assists"] = prj["assists"].apply(pd.to_numeric, errors="coerce")

# goals against average is a negative thing
prj["goals_against_average"] = -prj["goals_against_average"]

# reduce
prj = (
    prj
    [["name"] + SKATER_CATEGORIES + GOALIE_CATEGORIES]
    .groupby("name")
    .mean()
    .round(3)
    .reset_index()
)

prj.head()

,name,goals,assists,plus_minus,powerplay_points,shots_on_goal,hits,blocks,wins,save_percentage,goals_against_average,saves,shutouts
0,Aaron Ekblad,7.183,23.263,4.957,5.567,122.450,46.40,69.9,NaN,NaN,NaN,NaN,NaN
1,Adam Boqvist,7.400,16.300,-4.100,2.800,83.400,50.00,65.2,NaN,NaN,NaN,NaN,NaN
2,Adam Fox,6.683,24.720,12.823,9.053,103.113,29.55,74.0,NaN,NaN,NaN,NaN,NaN
3,Adam Gaudette,10.605,16.940,-2.340,5.275,67.865,31.10,21.5,NaN,NaN,NaN,NaN,NaN
4,Adam Henrique,16.880,15.453,-0.730,6.857,109.563,53.55,39.1,NaN,NaN,NaN,NaN,NaN


### Export

In [5]:
df = pd.merge(df, prj, how="inner", on="name")
df.head()

,team,name,position,goals,assists,plus_minus,powerplay_points,shots_on_goal,hits,blocks,wins,save_percentage,goals_against_average,saves,shutouts
0,Edm,Connor McDavid,C,30.810,53.447,3.337,23.950,177.233,25.65,22.5,NaN,NaN,NaN,NaN,NaN
1,Col,Nathan MacKinnon,C,28.867,45.260,8.037,21.863,255.060,36.20,20.8,NaN,NaN,NaN,NaN,NaN
2,Edm,Leon Draisaitl,"C,LW",33.977,48.113,-1.223,23.237,163.860,29.80,16.4,NaN,NaN,NaN,NaN,NaN
3,NYR,Artemi Panarin,LW,24.727,46.163,18.190,15.007,160.253,15.20,13.4,NaN,NaN,NaN,NaN,NaN
4,Was,Alex Ovechkin,LW,34.080,21.130,-0.617,15.117,245.010,143.45,22.4,NaN,NaN,NaN,NaN,NaN


In [6]:
df.to_csv("../data/projections-generated.csv", index=False)